In [1]:
import re
import string
import time
from copy import deepcopy

In [2]:
import pandas as pd
import os
import json

In [3]:
df = pd.read_excel('Quran.xlsx')

In [4]:
df.shape

(6236, 5)

In [5]:
df.head(10)

,surat,nama_surat,ayat,arabic,indonesian
0,1,Al-Fātiḥah,1,بِسْمِ اللَّهِ الرَّحْمَٰنِ الرَّحِيمِ,Dengan menyebut nama Allah Yang Maha Pemurah l...
1,1,Al-Fātiḥah,2,الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ,"Segala puji bagi Allah, Tuhan semesta alam."
2,1,Al-Fātiḥah,3,الرَّحْمَٰنِ الرَّحِيمِ,Maha Pemurah lagi Maha Penyayang.
3,1,Al-Fātiḥah,4,مَالِكِ يَوْمِ الدِّينِ,Yang menguasai di Hari Pembalasan.
4,1,Al-Fātiḥah,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ,"Hanya Engkaulah yang kami sembah, dan hanya ke..."
5,1,Al-Fātiḥah,6,اهْدِنَا الصِّرَاطَ الْمُسْتَقِيمَ,"Tunjukilah kami jalan yang lurus,"
6,1,Al-Fātiḥah,7,صِرَاطَ الَّذِينَ أَنْعَمْتَ عَلَيْهِمْ غَيْرِ...,(yaitu) Jalan orang-orang yang telah Engkau be...
7,2,Al-Baqarah,1,بِسْمِ اللَّهِ الرَّحْمَٰنِ الرَّحِيمِ الم,Alif laam miim.
8,2,Al-Baqarah,2,ذَٰلِكَ الْكِتَابُ لَا رَيْبَ ۛ فِيهِ ۛ هُدًى ...,Kitab (Al Quran) ini tidak ada keraguan padany...
9,2,Al-Baqarah,3,الَّذِينَ يُؤْمِنُونَ بِالْغَيْبِ وَيُقِيمُونَ...,"(yaitu) mereka yang beriman kepada yang ghaib,..."


In [6]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['email', 'percent', 'money', 'phone', 'user',
        'time', 'date', 'number'],
    # terms that will be annotated
    #annotate={"hashtag", "allcaps", "elongated", "repeated",'emphasis', 'censored'},
    annotate={"hashtag"},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

C:\Users\mmahf\miniconda3\envs\latihanstki\lib\site-packages\ekphrasis\classes\tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


C:\Users\mmahf\miniconda3\envs\latihanstki\lib\site-packages\ekphrasis\classes\exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [7]:
def bersih_data(text):
    return " ".join(text_processor.pre_process_doc(text))

In [8]:
# fungsi dari AMS 01-03. silakan cek bagaimana saya merubah menjadi fungsi

def non_ascii(text):
    return text.encode('ascii', 'replace').decode('ascii')

def remove_space_alzami(text):
    return " ".join(text.split())

def remove_emoji_alzami(text):
    return ' '.join(re.sub("([x#][A-Za-z0-9]+)"," ", text).split())

def remove_tab(text):
    return text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")

def remove_tab2(text):
    return re.sub('\s+',' ',text)

def remove_rt(text):
    return text.replace('RT'," ")

def remove_mention(text):
    return ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())

def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

def remove_excessive_dot(text):
    return text.replace('..'," ")

def change_stripe(text):
    return text.replace('-'," ")

def lower(text):
    return text.lower()

def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

def remove_excessive_dot(text):
    return text.replace('..'," ")

def lower(text):
    return text.lower()

def remove_whitespace_LT(text):
    return text.strip()

def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

def remove_punctuation(text):
    remove = string.punctuation
    remove = remove.replace("_", "") # don't remove hyphens
    pattern = r"[{}]".format(remove) # create the pattern
    return re.sub(pattern, "", text) 

In [9]:
# hapus untuk <>
def remove_number_eks(text):
    return text.replace('<number>'," ")

def remove_angka(text):
    return re.sub(r"\d+", "", text) 

def space_punctuation(text):
    return re.sub('(?<! )(?=[.,!?()])|(?<=[.,!?()])(?! )', r' ', text)

In [10]:
i = 0
final_string = []
s = ""
for text in df['indonesian'].values:
    filteredSentence = []
    EachReviewText = ""
    proc = remove_rt(text)
    proc = lower(proc)
    proc = change_stripe(proc)
    proc = remove_emoji_alzami(proc)
    proc = remove_tab(proc)
    proc = remove_tab2(proc)
    proc = non_ascii(proc)
    proc = remove_excessive_dot(proc)
    proc = remove_whitespace_LT(proc)
    proc = remove_whitespace_multiple(proc)
    proc = remove_single_char(proc)
    proc = space_punctuation(proc)
    proc = remove_punctuation(proc)
    proc = remove_space_alzami(proc)
    proc = bersih_data(proc)
    proc = remove_number_eks(proc)
    proc = remove_angka(proc) 
    EachReviewText = proc
    final_string.append(EachReviewText)

In [11]:
df["step01"] = final_string

In [12]:
df

,surat,nama_surat,ayat,arabic,indonesian,step01
0,1,Al-Fātiḥah,1,بِسْمِ اللَّهِ الرَّحْمَٰنِ الرَّحِيمِ,Dengan menyebut nama Allah Yang Maha Pemurah l...,dengan menyebut nama allah yang maha pemurah l...
1,1,Al-Fātiḥah,2,الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ,"Segala puji bagi Allah, Tuhan semesta alam.",segala puji bagi allah tuhan semesta alam
2,1,Al-Fātiḥah,3,الرَّحْمَٰنِ الرَّحِيمِ,Maha Pemurah lagi Maha Penyayang.,maha pemurah lagi maha penyayang
3,1,Al-Fātiḥah,4,مَالِكِ يَوْمِ الدِّينِ,Yang menguasai di Hari Pembalasan.,yang menguasai di hari pembalasan
4,1,Al-Fātiḥah,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ,"Hanya Engkaulah yang kami sembah, dan hanya ke...",hanya engkaulah yang kami sembah dan hanya kep...
...,...,...,...,...,...,...
6231,114,An-Nās,2,مَلِكِ النَّاسِ,Raja manusia.,raja manusia
6232,114,An-Nās,3,إِلَٰهِ النَّاسِ,Sembahan manusia.,sembahan manusia
6233,114,An-Nās,4,مِنْ شَرِّ الْوَسْوَاسِ الْخَنَّاسِ,Dari kejahatan (bisikan) syaitan yang biasa be...,dari kejahatan bisikan syaitan yang biasa bers...
6234,114,An-Nās,5,الَّذِي يُوَسْوِسُ فِي صُدُورِ النَّاسِ,yang membisikkan (kejahatan) ke dalam dada man...,yang membisikkan kejahatan ke dalam dada manusia


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6236 entries, 0 to 6235
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   surat       6236 non-null   int64 
 1   nama_surat  6236 non-null   object
 2   ayat        6236 non-null   int64 
 3   arabic      6236 non-null   object
 4   indonesian  6236 non-null   object
 5   step01      6236 non-null   object
dtypes: int64(2), object(4)
memory usage: 292.4+ KB


In [14]:
# token
import nltk
from nltk.tokenize import word_tokenize 

In [15]:
def word_tokenize_wrapper(text):
  return word_tokenize(text)

In [16]:
df['tokens'] = df['step01'].apply(word_tokenize_wrapper)

In [17]:
df.head(10)

,surat,nama_surat,ayat,arabic,indonesian,step01,tokens
0,1,Al-Fātiḥah,1,بِسْمِ اللَّهِ الرَّحْمَٰنِ الرَّحِيمِ,Dengan menyebut nama Allah Yang Maha Pemurah l...,dengan menyebut nama allah yang maha pemurah l...,"[dengan, menyebut, nama, allah, yang, maha, pe..."
1,1,Al-Fātiḥah,2,الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ,"Segala puji bagi Allah, Tuhan semesta alam.",segala puji bagi allah tuhan semesta alam,"[segala, puji, bagi, allah, tuhan, semesta, alam]"
2,1,Al-Fātiḥah,3,الرَّحْمَٰنِ الرَّحِيمِ,Maha Pemurah lagi Maha Penyayang.,maha pemurah lagi maha penyayang,"[maha, pemurah, lagi, maha, penyayang]"
3,1,Al-Fātiḥah,4,مَالِكِ يَوْمِ الدِّينِ,Yang menguasai di Hari Pembalasan.,yang menguasai di hari pembalasan,"[yang, menguasai, di, hari, pembalasan]"
4,1,Al-Fātiḥah,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ,"Hanya Engkaulah yang kami sembah, dan hanya ke...",hanya engkaulah yang kami sembah dan hanya kep...,"[hanya, engkaulah, yang, kami, sembah, dan, ha..."
5,1,Al-Fātiḥah,6,اهْدِنَا الصِّرَاطَ الْمُسْتَقِيمَ,"Tunjukilah kami jalan yang lurus,",tunjukilah kami jalan yang lurus,"[tunjukilah, kami, jalan, yang, lurus]"
6,1,Al-Fātiḥah,7,صِرَاطَ الَّذِينَ أَنْعَمْتَ عَلَيْهِمْ غَيْرِ...,(yaitu) Jalan orang-orang yang telah Engkau be...,yaitu jalan orang orang yang telah engkau beri...,"[yaitu, jalan, orang, orang, yang, telah, engk..."
7,2,Al-Baqarah,1,بِسْمِ اللَّهِ الرَّحْمَٰنِ الرَّحِيمِ الم,Alif laam miim.,alif laam miim,"[alif, laam, miim]"
8,2,Al-Baqarah,2,ذَٰلِكَ الْكِتَابُ لَا رَيْبَ ۛ فِيهِ ۛ هُدًى ...,Kitab (Al Quran) ini tidak ada keraguan padany...,kitab al quran ini tidak ada keraguan padanya ...,"[kitab, al, quran, ini, tidak, ada, keraguan, ..."
9,2,Al-Baqarah,3,الَّذِينَ يُؤْمِنُونَ بِالْغَيْبِ وَيُقِيمُونَ...,"(yaitu) mereka yang beriman kepada yang ghaib,...",yaitu mereka yang beriman kepada yang ghaib ya...,"[yaitu, mereka, yang, beriman, kepada, yang, g..."


In [18]:
normalized_word = pd.read_csv('kamus_clean.csv')

In [19]:
normalized_word_dict = {}

for index, row in normalized_word.iterrows():
    if row[0] not in normalized_word_dict:
        normalized_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalized_word_dict[term] if term in normalized_word_dict else term for term in document]
    

C:\Users\mmahf\AppData\Local\Temp\ipykernel_23604\2492408998.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[0] not in normalized_word_dict:
C:\Users\mmahf\AppData\Local\Temp\ipykernel_23604\2492408998.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  normalized_word_dict[row[0]] = row[1]


In [20]:
df['final_tokens'] = df['tokens'].apply(normalized_term)

In [21]:
i=0
final_string_tokens = []
for text in df['final_tokens'].values:
    EachReviewText = ""
    EachReviewText = ' '.join(text)
    final_string_tokens.append(EachReviewText)

In [22]:
df["step02"] = final_string_tokens

In [23]:
df

,surat,nama_surat,ayat,arabic,indonesian,step01,tokens,final_tokens,step02
0,1,Al-Fātiḥah,1,بِسْمِ اللَّهِ الرَّحْمَٰنِ الرَّحِيمِ,Dengan menyebut nama Allah Yang Maha Pemurah l...,dengan menyebut nama allah yang maha pemurah l...,"[dengan, menyebut, nama, allah, yang, maha, pe...","[dengan, menyebut, nama, allah, yang, maha, pe...",dengan menyebut nama allah yang maha pemurah l...
1,1,Al-Fātiḥah,2,الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ,"Segala puji bagi Allah, Tuhan semesta alam.",segala puji bagi allah tuhan semesta alam,"[segala, puji, bagi, allah, tuhan, semesta, alam]","[segala, puji, bagi, allah, tuhan, semesta, alam]",segala puji bagi allah tuhan semesta alam
2,1,Al-Fātiḥah,3,الرَّحْمَٰنِ الرَّحِيمِ,Maha Pemurah lagi Maha Penyayang.,maha pemurah lagi maha penyayang,"[maha, pemurah, lagi, maha, penyayang]","[maha, pemurah, lagi, maha, penyayang]",maha pemurah lagi maha penyayang
3,1,Al-Fātiḥah,4,مَالِكِ يَوْمِ الدِّينِ,Yang menguasai di Hari Pembalasan.,yang menguasai di hari pembalasan,"[yang, menguasai, di, hari, pembalasan]","[yang, menguasai, di, hari, pembalasan]",yang menguasai di hari pembalasan
4,1,Al-Fātiḥah,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ,"Hanya Engkaulah yang kami sembah, dan hanya ke...",hanya engkaulah yang kami sembah dan hanya kep...,"[hanya, engkaulah, yang, kami, sembah, dan, ha...","[hanya, engkaulah, yang, kami, sembah, dan, ha...",hanya engkaulah yang kami sembah dan hanya kep...
...,...,...,...,...,...,...,...,...,...
6231,114,An-Nās,2,مَلِكِ النَّاسِ,Raja manusia.,raja manusia,"[raja, manusia]","[raja, manusia]",raja manusia
6232,114,An-Nās,3,إِلَٰهِ النَّاسِ,Sembahan manusia.,sembahan manusia,"[sembahan, manusia]","[sembahan, manusia]",sembahan manusia
6233,114,An-Nās,4,مِنْ شَرِّ الْوَسْوَاسِ الْخَنَّاسِ,Dari kejahatan (bisikan) syaitan yang biasa be...,dari kejahatan bisikan syaitan yang biasa bers...,"[dari, kejahatan, bisikan, syaitan, yang, bias...","[dari, kejahatan, bisikan, syaitan, yang, bias...",dari kejahatan bisikan syaitan yang biasa bers...
6234,114,An-Nās,5,الَّذِي يُوَسْوِسُ فِي صُدُورِ النَّاسِ,yang membisikkan (kejahatan) ke dalam dada man...,yang membisikkan kejahatan ke dalam dada manusia,"[yang, membisikkan, kejahatan, ke, dalam, dada...","[yang, membisikkan, kejahatan, ke, dalam, dada...",yang membisikkan kejahatan ke dalam dada manusia


In [24]:
df.to_excel('clean_dataset_quran.xlsx')